# Joye-Libert Secure Aggregation Scheme (JL)

In [41]:
from protocols.buildingblocks.JoyeLibert import JLS
import random

t = 412
nclients = 10
keysize = 2048 
inputsize = 16

dimension = 1 
JL = JLS(nclients, inputsize, dimension, keysize)

### Generate Keys for $nclients and server

In [42]:
# generate keys
pp, skey, ukey = JL.generate_keys()

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=10028...99601, H(x)=0xbe4bae0)>
 - Aggregator Key: <ServerKey -0x1fa53b6>
 - User Keys: {0: <UserKey 0x6e4a71ee>, 1: <UserKey 0x2073bf79>, 2: <UserKey 0x1c0520fb>, 3: <UserKey 0x49664e21>, 4: <UserKey 0x11ffc180>, 5: <UserKey 0x8e1fd3c1>, 6: <UserKey 0x4956aa37>, 7: <UserKey 0xa1f837ed>, 8: <UserKey 0x14b0df20>, 9: <UserKey 0xb0b242b2>}


### Working with integers: Encrypt random numbers with client keys and aggregate them. Then decrypt them with server key

In [43]:
# generate $nclients random numbers
l =[]
for i in range(0,nclients):
    l.append(random.randint(0,2**16))

print("Clients Inputs: ", l )

# encrypt each number with a different client key
e=[]
for i in range(0,nclients):
    e.append(ukey[i].encrypt(l[i],t))

# aggregate all ciphetexts 
c = JL.aggregate(e)
print("Aggregated Ciphetext", c)



# decrypt the aggregate
s = skey.decrypt(c,t)
print("Sum=", s )

# check if the result is correct
print("Verify=", s == sum(l))

Clients Inputs:  [3876, 61514, 5699, 40439, 51589, 22328, 22097, 29745, 1612, 26151]
Aggregated Ciphetext <EncryptedNumber 95898...01958>
Sum= 265050
Verify= True


### Working with vectors: Encrypt random vectors with client keys and aggregate them. Then decrypt them with server key

In [44]:
vsize = 20
JL.setinputdimension(vsize)

# generate $nclients random vectors
L = []
for _ in range(nclients):
    l =[]
    for i in range(0,vsize):
        l.append(random.randint(0,1000))
    L.append(l)

# encrypt all ciphers
E=[]
for i in range(0,nclients):
    E.append(ukey[i].encrypt_vector(L[i],t))

# aggregate them
C = JL.aggregate_vector(E)
print("Aggregated Ciphetext", C)

# decrypt the aggregate
s = skey.decrypt_vector(C,t)

summ=L[0]
from operator import add
for l in L[1:]:
    summ = list(map(add, summ, l))


# check if the result is correct
print("Sum = ", s)
print("Verify: ", s == summ)

Aggregated Ciphetext [<EncryptedNumber 76185...53378>]
Sum =  [4685, 4724, 4266, 4620, 4612, 6221, 3315, 5778, 6312, 6235, 6471, 3871, 4735, 6140, 3820, 5963, 5743, 6003, 5028, 4254]
Verify:  True


# Shamir Secret Sharing (SS)

This is demo for our implementation of Shamir's secret sharing. We choose the Field $\mathbb{Z}_p$ where $p > 2^\lambda$ is a prime ($\lambda$ is the key size)

In [45]:
from protocols.buildingblocks.ShamirSS import SSS
import random

threshold = 70
nclients  = 100
SS = SSS(1024)

number1 = random.randint(0,2**16-1)
number2 = random.randint(0,2**16-1)

allshares1 = SS.share(threshold, nclients, number1)
allshares2 = SS.share(threshold, nclients, number2)

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

lagcoef = SS.lagrange(shares1)
reconstucted1 =  SS.recon(shares1, lagcoef)
reconstucted2 =  SS.recon(shares2, lagcoef)
reconstucted =  SS.recon(shares, lagcoef)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Deffie-Hellman Key Aggreement (KA)
It uses ECDH p256 curve. It then computes with SHA256 over the ECDH agreed key to generate a secret key for AES (256 bit)

In [46]:
from protocols.buildingblocks.KeyAggreement import KAS

sharedkeysize = 1024

KA1 = KAS()
KA2 = KAS()

KA1.generate()
KA2.generate()

# check if the shared secrets match
print("Verify:", KA1.agree(KA2.pk, sharedkeysize) == KA2.agree(KA1.pk, sharedkeysize))

# print shared key
sharedkey = KA1.agree(KA2.pk, sharedkeysize)
print("key: " + str(sharedkey))

Verify: True
key: 28760237286762969710832973705079029447928951039879689540155201639491404899121495082940115174469358155882193453747373158780011329314305022653705243267263043993397984956958960100891532634612269213379425175057641020202716259506704432177380792057325528157743950719508063416408111240912626771550211181843474908193


In [47]:
# test serializing secret keys
sk1_bytes = KA1.get_sk_bytes()
KA = KAS()
KA.generate_from_bytes(sk1_bytes)
print("Verify:", KA.sk == KA1.sk and  KA.pk == KA1.pk)

Verify: True


# AES-GCM Authenticated Encryption (AE)

In [48]:
import protocols.buildingblocks.AESGCM128 as aes

message=b"TOP SECRET"

key = aes.EncryptionKey(sharedkey)
e = key.encrypt(message)
m = key.decrypt(e)
print("Verify:", message == m)

Verify: True


# Vector Encoding Scheme (VE)
This is a vector to vector encoding. It batches several elements of the vectors into a single integer. The batch size depends on the size of that integer (i.e. this means the plaintext size of an encryption scheme). It also depends on the number of safety bits need to be preserved for each vector element (this is to support addition operation without reaching an overflow)

In [49]:
from protocols.buildingblocks.VectorEncoding import VES

keysize = 1024
nclients = 100
valuesize = 16
vectorsize = 100

VE = VES(keysize, nclients, valuesize, vectorsize)
vector = list(range(1,vectorsize+1))

# encode the vector 
E = VE.encode(V=vector)
print("encoded decreased the vector size from {} to {}".format(len(vector),len(E)))
# decode the vector 
V = VE.decode(E=E)
print("Verify:", V == vector)

encoded decreased the vector size from 100 to 3
Verify: True


# Function Secret Sharing (FSS)
### Function secret sharing is a scheme to share a function $f_s(x): H(x)^s$
The value $s$ is a secret the defines the secret function. It is first shared using Shamir secret sharing. Then, each share holder (for a given $x$) can evaluate a partial result of the function $f_s(x)$. The partial results can be combined to contruct the value of $H(x)^s$.

This demo shows how can we share a secret function and recompute its results in a distributed way. It also shows the homomorphic property: $f_{s_1+s_2}(x) = f_{s_1}(x) \cdot f_{s_2}(x)$. Specifically, by summing the shares of two secret functions, we can evaluate the result of the multiplication of these functions

In [50]:
from protocols.buildingblocks.FunctionSS import FSS, FShare
from protocols.buildingblocks.JoyeLibert import JLS
from protocols.buildingblocks.utils import powmod
import random
from math import factorial

threshold = 70
nclients  = 100
inputsize = 16
dimension = 1
keylength = 1024

JL = JLS(nclients, inputsize, dimension, keylength)
pp,_,_ = JL.generate_keys()

In [51]:
# create two secret functions
s1 = random.SystemRandom().getrandbits(keylength) 
s2 = random.SystemRandom().getrandbits(keylength)
s = s1+s2

# initialize the FSS scheme
FS = FSS(keylength, pp.n)


# share two secrets
allshares1 = FS.share(threshold, nclients, s1)
allshares2 = FS.share(threshold, nclients, s2)

# pick a threshold number of shares randomly 
random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

# compute the sum of the picked shares 
shares = [x+y for x,y in zip(shares1, shares2)]

In [52]:

# pick a random t to evaluate the function on
t = random.randint(0,2**16-1)

# evaluate the function on t: f_s(t)  using each share
evalsahres1 = [s.eval(t) for s in shares1]
evalsahres2 = [s.eval(t) for s in shares2]
evalsahres = [s.eval(t) for s in shares]

delta = factorial(nclients)

# reconstruct the results of f_s(t) 
lagcoef = FS.lagrange(shares1, delta)
reconstucted1 =  FS.recon(evalsahres1, delta, lagcoef)
reconstucted2 =  FS.recon(evalsahres2, delta, lagcoef)
reconstucted =  FS.recon(evalsahres, delta, lagcoef)

# check if the reconstructed values match the result of function f_s(t)

print("Verify f_s1(t): ", powmod(FShare.H(t), s1 * delta,pp.nsquare)  == reconstucted1 )
print("Verify f_s2(t): ", powmod(FShare.H(t), s2 * delta,pp.nsquare) == reconstucted2)
print("Verify f_{s1+s2}(t): ", powmod(FShare.H(t), (s1+s2) * delta,pp.nsquare) == reconstucted)


Verify f_s1(t):  True
Verify f_s2(t):  True
Verify f_{s1+s2}(t):  True


### Vector Function Secret Sharing (VFSS)
This is the same as FSS except that we share a function that outputs a vector $g_s(x, v) : [H(x||0)^s, H(x||1)^s, ...,  H(x||v)^s]$ 

In [53]:
vsize = 10

# pick a random t to evaluate the function on
t = random.randint(0,2**16-1)

# evaluate the function on t: f_s(t)  using each share
evalsahresv1 = [s.eval_vector(t, vsize) for s in shares1]
evalsahresv2 = [s.eval_vector(t, vsize) for s in shares2]
evalsahresv = [s.eval_vector(t, vsize) for s in shares]

delta = factorial(nclients)

# reconstruct the results of f_s(t) 
lagcoef = FS.lagrange(shares1, delta)
reconstucted1 =  FS.recon_vector(evalsahresv1, delta, lagcoef)
reconstucted2 =  FS.recon_vector(evalsahresv2, delta, lagcoef)
reconstucted =  FS.recon_vector(evalsahresv, delta, lagcoef)

# check if the reconstructed values match the result of function f_s(t)
valid = True
Y1 = []
Y2 = []
Y = []
for i in range(len(reconstucted1)):
    h = FShare.H((i << keylength // 4) | t)
    Y1.append(powmod(h, s1 * delta,pp.nsquare))
    Y2.append(powmod(h, s2 * delta,pp.nsquare))
    Y.append(powmod(h, s * delta,pp.nsquare))

print("Verify f_s1(t): ", Y1  == reconstucted1 )
print("Verify f_s2(t): ", Y2 == reconstucted2)
print("Verify f_{s1+s2}(t): ", Y == reconstucted)


Verify f_s1(t):  True
Verify f_s2(t):  True
Verify f_{s1+s2}(t):  True


# Pseudo Randomg Generator (PRG)
## We use AES-CTR to generate a pseudo random vector from a seed

The demo first generates a random seed. The seed is used as the AES key to encrypt a '0x00'-repeated message of large size. The encryption result is transformed into a vector of integers. The output vector size is controlled by the size of the input message.

In [54]:
from protocols.buildingblocks.PRG import PRG
import random

securitybits = 128 #bits
elementsize = 16 #bits
vectorsize = 2**20 #elements

# setup the PRG 
prg = PRG(vectorsize, elementsize)

In [55]:
# get a random seed of 16 bytes long
b = random.SystemRandom().getrandbits(securitybits).to_bytes(securitybits // 8,"big")

# generate a random vector from that seed 
B = prg.eval(b)

print("Generated a pseudo random vector of length {} ({} bits elements) from {} bits random value".format(len(B), elementsize, securitybits))
B

Generated a pseudo random vector of length 1048576 (16 bits elements) from 128 bits random value


[2581,
 8651,
 36944,
 21858,
 5246,
 27384,
 19215,
 45468,
 27483,
 55546,
 34947,
 26068,
 28509,
 59154,
 36210,
 56027,
 35402,
 15306,
 11968,
 47452,
 10736,
 32201,
 32610,
 5252,
 27261,
 23794,
 32604,
 18579,
 7424,
 9276,
 63891,
 44801,
 60307,
 63772,
 820,
 46030,
 46441,
 15753,
 43667,
 44822,
 39463,
 38945,
 19163,
 60317,
 34155,
 46490,
 64946,
 49711,
 8128,
 32145,
 31529,
 7959,
 29481,
 34549,
 29188,
 25435,
 32596,
 12101,
 16086,
 38184,
 11695,
 6754,
 22245,
 409,
 24791,
 36982,
 985,
 7021,
 3846,
 13090,
 44824,
 29824,
 61550,
 45414,
 18003,
 61650,
 17219,
 3710,
 49754,
 50474,
 25330,
 21764,
 43694,
 6045,
 22398,
 31636,
 10415,
 20622,
 64705,
 10220,
 24466,
 47426,
 65065,
 29506,
 10046,
 20724,
 8720,
 64768,
 32251,
 24441,
 49752,
 17645,
 34267,
 46841,
 56271,
 40673,
 65075,
 14492,
 32133,
 7120,
 38348,
 64202,
 37153,
 44950,
 29001,
 754,
 6618,
 53778,
 2738,
 1577,
 2390,
 56053,
 48158,
 29473,
 60779,
 63969,
 20647,
 58041,
 51